In [1]:
import wandb

# Log in to WandB
wandb.login()

# Replace with your entity, project, and run ID
entity = "nlp_ls"
project = "dspro2-predicting-region"
run_ids = ["8my0kcc5", "l964tv8w", "jzr47aha", "064k6n6n", "11yf08nx"]

# Value to add to the summary
summary_key = "test_data_run_id"
summary_value = "w1098m89"

# Access the run
api = wandb.Api()
for run_id in run_ids:
    run = api.run(f"{entity}/{project}/{run_id}")

    # Update the summary
    run.summary[summary_key] = summary_value
    run.summary.update()

    print(f"Successfully updated summary of run {run_id} with {summary_key}: {summary_value}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: killusions. Use `wandb login --relogin` to force relogin


Successfully updated summary of run 8my0kcc5 with test_data_run_id: w1098m89
Successfully updated summary of run l964tv8w with test_data_run_id: w1098m89
Successfully updated summary of run jzr47aha with test_data_run_id: w1098m89
Successfully updated summary of run 064k6n6n with test_data_run_id: w1098m89
Successfully updated summary of run 11yf08nx with test_data_run_id: w1098m89
